In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [3]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)

In [6]:
for l in input_text:
    if l == '':
        print(input_text.index(l))
        ind = input_text.index(l) 

input_ops = input_text[:ind] 
input_data = input_text[ind+1:]

506


In [15]:
## store parts as named tuples
part = collections.namedtuple('part', 'x m a s')
parts = []
for l in input_data:
    tmp = l.lstrip('{').rstrip('}').split(',')
    assert len(tmp) == 4 
    assert tmp[0].startswith('x=') and tmp[1].startswith('m=') and tmp[2].startswith('a=') and tmp[3].startswith('s=')
    tmp = [int(x.split('=')[1]) for x in tmp]
    parts.append(part(*tmp))

In [74]:
def generic_op(str_op):
    '''given a str (like "a<123:R" or "A"), return a function that takes a part and returns True/False + outcome'''
    if len(str_op) == 1:
        if str_op == 'R':
            return (lambda x: True, 'R')
        elif str_op == 'A':
            return (lambda x: True, 'A')
        else:
            assert False, 'unknown op'
    elif len(str_op) > 1 and ('<' in str_op or '>' in str_op):
        tmp = str_op.split(':')
        assert len(tmp) == 2, f'{str_op}'
        cond = tmp[0]
        result = tmp[1]
        assert cond[1] in ['<', '>'], f'{cond} not < >'
        operation = cond[1]
        cond = cond.split(operation)
        assert len(cond) == 2
        component = cond[0]
        value = int(cond[1])
        assert len(component) == 1 and component in ['x', 'm', 'a', 's']
        if operation == '<':
            return lambda x: getattr(x, component) < value, result
        elif operation == '>':
            return lambda x: getattr(x, component) > value, result
        else:
            assert False, 'unknown operation'
    else:   ## box name 
        return (lambda x: True, str_op)

def evaluate_op(str_op, part):
    '''given a str_op and part, evaluate op and return result if True'''
    op_fun, result_if_true = generic_op(str_op)
    # print(op_fun(part), result_if_true)
    if op_fun(part):
        return result_if_true 
    else:
        return None 
    
def evaluate_box(box_name, part):
    '''loop through ops in box and evaluate until result is found. recursively call other boxes if needed. assert false if end is reached without result'''
    ops = dict_boxes[box_name]
    for op in ops:
        res = evaluate_op(op, part)
        # print(op, res)
        if res is not None:
            if res in ['A', 'R']:
                return res 
            else: 
                return evaluate_box(res, part)    
    assert False, 'no result found'
    
## create dict with boxes with lsit of str_ops
dict_boxes = {} 
for l in input_ops:
    tmp = l.split('{')
    assert len(tmp) == 2, f'{l}'
    name_box = tmp[0]
    all_ops = tmp[1].rstrip('}').split(',')
    assert name_box not in dict_boxes
    dict_boxes[name_box] = [x for x in all_ops]


In [75]:
total_accepted = 0
for part_curr in parts:
    res = evaluate_box('in', part_curr)
    if res == 'A':
        total_accepted += (part_curr.x + part_curr.m + part_curr.a + part_curr.s) 

print(total_accepted)

353046


## part 2: 

excercise: of all possible parts with x,m,a,s in (1, 4000), how many combinations are possible?

bit like day 5, which I solved very *hackily* at the time.. 

**Thoughts:**
- Want to solve with ranges. Seems doable.. there is 1 condition per time that needs to be evaluated. For example, if the first one is `{a<3:nz,A}` then `x,m,a<3,s` go to `nz` and `x,m,a>=3,s` are `A`.
- So want to do the same as before, but let the function take ranges as input. Split up (recursively?) at each condition, save all ranges that end up at `A`.
- So conceptually it's not too difficult, but I fear that I would need to write new functions .. Instead of evaluating the ints in a part, I need to determine the split from the str_op and return two new ranges and their destinations. 

In [149]:
part_range = collections.namedtuple('part_range', 'x_min x_max m_min m_max a_min a_max s_min s_max')

def split_range(str_op, curr_range):
    '''Split range according do str_op. Return list of new ranges + destination'''
    ## if str_op has no condition (box, A, R), return curr_range + destination 
    if str_op in ['A', 'R']:
        return [(curr_range, str_op)]
    elif ('<' in str_op or '>' in str_op):  ## if str_op has condition, split curr_range into 2 and return 2 new ranges + destination
        ## unpack str_op
        tmp = str_op.split(':')
        assert len(tmp) == 2, f'{str_op}'
        cond = tmp[0]
        result = tmp[1]
        assert cond[1] in ['<', '>'], f'{cond} not < >'
        operation = cond[1]
        cond = cond.split(operation)
        assert len(cond) == 2
        component = cond[0]
        value = int(cond[1])
        assert len(component) == 1 and component in ['x', 'm', 'a', 's']

        ## split curr_range
        comp_min = getattr(curr_range, f'{component}_min')
        comp_max = getattr(curr_range, f'{component}_max')
        
        if operation == '<':
            if comp_max < value:  ## if max is less than value, then entire range is accepted 
                return [(curr_range, result)]
            elif comp_min >= value:  ## if min is greater than value, then entire range is rejected
                return [(curr_range, None)]
            else:  ## if min is less than value and max is greater than value, then split range into 2
                new_range_true = curr_range._replace(**{f'{component}_max': value - 1})
                new_range_false = curr_range._replace(**{f'{component}_min': value})
                return [(new_range_true, result), (new_range_false, None)]
            
        elif operation == '>':
            if comp_min > value: 
                return [(curr_range, result)]
            elif comp_max <= value:
                return [(curr_range, None)]
            else: 
                new_range_true = curr_range._replace(**{f'{component}_min': value + 1})
                new_range_false = curr_range._replace(**{f'{component}_max': value})
                return [(new_range_true, result), (new_range_false, None)]
    else:
        return [(curr_range, str_op)]
    
def evaluate_box_range(box_name, curr_range):
    assert str(type(curr_range)) == "<class '__main__.part_range'>", 'only accept a single range'
    if box_name == 'A':  ## if box is A, return total number of values in range
        return ((curr_range.x_max - curr_range.x_min + 1) * (curr_range.m_max - curr_range.m_min + 1) * (curr_range.a_max - curr_range.a_min + 1) * (curr_range.s_max - curr_range.s_min + 1))
    elif box_name == 'R':  ## if box is R, return 0
        return 0
    else:
        total = 0  ## collect total from all sub-boxes
        ops = dict_boxes[box_name]
        n_ops = len(ops)
        i_op = 0  # start at first op 
        list_new_ranges = [curr_range]  # ranges to be evaluated (until they reach A, R, or box (via splits)) 
        list_final_ranges = []  # ranges that have reached A or R or box 
        while len(list_new_ranges) > 0:
            tmp_curr_range = list_new_ranges.pop()
            tmp_new_range_results = split_range(ops[i_op], tmp_curr_range)
            
            for tmp_nr_result in tmp_new_range_results:
                if tmp_nr_result[1] is not None:  # if A, R or box, add to list_final_ranges
                    list_final_ranges.append(tmp_nr_result)
                else:  # if not, add back to list_new_ranges and evaluate in next loop with next op 
                    list_new_ranges.append(tmp_nr_result[0])
            i_op += 1  # move to next op
            if i_op == n_ops and len(list_new_ranges) > 0:
                assert False, 'no result found'
        
        ## count total of all sub boxes 
        for tmp_range in list_final_ranges:
            total += evaluate_box_range(tmp_range[1], tmp_range[0])
        
        return total

In [150]:
start_range = part_range(1, 4000, 1, 4000, 1, 4000, 1, 4000)
evaluate_box_range('in', start_range)

125355665599537